# Earth Lab Capstone Project: Where can soil moisture improve rainfall-triggered landslide predictability?

## Author: Jacquelyn Witte

## This Notebook exports data from SMAP and ESA CCI soil moisture and GPM daily and IMERG 30min precipitation co-located to Landslides in the US

- Based on 2015-2020 Landslide events from the NASA Global Landslide Catalog (GLC)
- Using Landslide locations over Colorado as a workflow example
- Workflow can be applied to any USA state defined in the GLC

In [1]:
# Override saving plots
GLOBAL_CACHE_OVERRIDE = True

In [2]:
import os
from glob import glob

import datetime as dt
import earthpy as et
import h5py
import numpy as np
import pandas as pd
import re
import xarray as xr

import warnings
warnings.filterwarnings('ignore')

In [3]:
def read_smap(filepath, index):
    """
    Reads SMAP data and returns the variable of interest.
    
    Parameters
    ----------
    filepath: Str
        File path of a SMAP L3 HDF5 file
        
    group_id: String
        Groups within the file to access
        
    index: int
        Index associated with the variable to retrieve
    
    Returns
    -------
    data: 2D numpy.ndarray (lat, lon)
    date: Date String yyyymmdd
    """
    # Read the file
    group_id = 'Soil_Moisture_Retrieval_Data_PM'
    with h5py.File(filepath, 'r') as f:
        # Extract data info
        data_id = list(f[group_id].keys())[index]
        data = f[group_id][data_id][:,:]
        data[data == f[group_id][data_id].attrs['_FillValue']] = np.nan
        
        filename = os.path.basename(filepath)
        yyyymmdd= filename.split('_')[5]
        yyyy = int(yyyymmdd[0:4])
        mm = int(yyyymmdd[4:6])
        dd = int(yyyymmdd[6:8])
        date=dt.datetime(yyyy,mm,dd)
    return data, date

In [4]:
def findfile(input_files, input_date):
    """
    Returns a single file from a list of files.
    
    Parameters
    ----------
    input_files: List of strings
        List of full path to the file
        
    input_date: String
        YYYYMMDD format
        
    Returns
    -------
    file: Str
    """
    file = [x for x in input_files if re.findall(input_date, x)]
    if not file:
        raise ValueError('File does not exist for '+input_date)
    return file

In [5]:
def nearestneighbor_ncdf(input_file, parameter, loc):
    """
    Extracts nearest neighbor value based on location and desired parameter. 
    
    Parameters
    ----------   
    input_file: Str - full path to a single file
    
    parameter: Str 
    
    loc: tuple (degree longtitude, degree latitude)
    
    Returns
    -------
    float
    """
    # read the netcdf file
    try:
        data_xr = xr.open_dataset(input_file).squeeze()
    except IOError:
        print("This file is not accessible: "+input_file)
    finally:
        data_xr.close()
    
    # subset the file
    res = data_xr[parameter].sel(indexers={
            'lon': loc[0],
            'lat': loc[1]},
            method="nearest")
    
    return float(res.values)

In [6]:
def get_imerg_hires(imerg_files, glc_df):
    """
    Reads all IMERG 30min CSV file into a dataFrame.
    
    # Ref: https://www.geeksforgeeks.org/ways-to-filter-pandas-dataframe-by-column-values/

    Parameters
    ----------
    imerg_files: List of strings
        List of full path to the file
    
    glc_df: dataFrame
        Global Landslide Catalog 
    
    Returns
    -------
    imerg: dataFrame
        Contains datetime, landslide ID, precipitation

    """
    id_list = glc_df['event_id'].values.tolist()
    
    list = []
    for f in imerg_files:
        print(f)
        temp_df = pd.read_csv(f)
        # filter for landslide id
        list.append(temp_df[temp_df['id'].isin(id_list)])

    imerg = pd.concat(list)
    # convert datetime to pd datetime because some dates are not in the right format
    imerg['datetime'] = pd.to_datetime(imerg['datetime'])
    # Create a simple date string to compare with the GLC data
    imerg['yyyymmdd'] = pd.to_datetime(imerg['datetime']).dt.strftime('%Y%m%d')
    #imerg.index = pd.to_datetime(imerg.index)
    imerg = imerg.reset_index().set_index('datetime')
    return imerg

### Choose the US state 

In [7]:
westernUS = ['Colorado', 'Utah', 'Idaho',
             'California', 'Oregon', 'Washington']
state = westernUS[4]

### Read and subset to Landslides >= year 2015 (SMAP data starts in 2015)
- Based on the state chosen

In [8]:
# Read the GLC file
glc_file = os.path.join(et.io.HOME,
                        'earth-analytics',
                        'data', 'capstone', 
                        'landslide', 'nasa_global_landslide_catalog_point.csv')

glc = pd.read_csv(glc_file)

glc_state = glc[(glc['country_code'] == 'US') 
             & (glc['admin_division_name'] == state) 
             & (glc['landslide_trigger'] != 'freeze_thaw') 
             & (glc['landslide_trigger'] != 'snowfall_snowmelt') 
             & (glc['landslide_trigger'] != 'earthquake') 
             & (glc['landslide_trigger'] != 'leaking_pipe') 
             & (glc['landslide_trigger'] != 'no_apparent_trigger') 
             & (glc['landslide_trigger'] != 'other')              
             & (glc['landslide_trigger'] != 'unknown')
            ]
# convert to pandas datetime
glc_state['date'] = pd.to_datetime(glc_state['event_date'])
glc_state = glc_state.set_index('date').sort_index()
glc_state_gt2015 = glc_state[glc_state.index > '2015-04-01']

print(glc_state_gt2015.shape)
print(np.unique(glc_state_gt2015['landslide_trigger']))
print(np.unique(glc_state_gt2015['landslide_category']))
print(glc_state_gt2015.columns)

# Dropping last dataframe if state=Utah - SMAP data does not exist
if state == 'Utah':
    glc_state_gt2015 = glc_state_gt2015.drop(pd.to_datetime(
                                                        '2019-06-26 04:00:00'))

glc_state_gt2015.tail()

(71, 31)
['continuous_rain' 'downpour' 'flooding' 'rain']
['debris_flow' 'landslide' 'mudslide' 'rock_fall' 'rotational_slide']
Index(['OBJECTID', 'Shape', 'source_name', 'source_link', 'event_id',
       'event_date', 'event_time', 'event_title', 'event_description',
       'location_description', 'location_accuracy', 'landslide_category',
       'landslide_trigger', 'landslide_size', 'landslide_setting',
       'fatality_count', 'injury_count', 'storm_name', 'photo_link',
       'comments', 'event_import_source', 'event_import_id', 'latitude',
       'longitude', 'country_name', 'country_code', 'admin_division_name',
       'gazetteer_closest_point', 'gazetteer_distance', 'submitted_date',
       'last_edited_date'],
      dtype='object')


,OBJECTID,Shape,source_name,source_link,event_id,event_date,event_time,event_title,event_description,location_description,...,event_import_id,latitude,longitude,country_name,country_code,admin_division_name,gazetteer_closest_point,gazetteer_distance,submitted_date,last_edited_date
date,,,,,,,,,,,,,,,,,,,,,
2018-01-27 18:00:00,7979511,"(-123.89150319899994, 45.56208131200003)",KREM Channel 2,http://www.krem.com/news/one-lane-of-highway-1...,11227,2018-01-27 18:00:00,18:00,Hwy 101 south Garibaldi Landslide,400-500 cubic yards of rock and mud came down ...,"Milepost 57, US-101, Bay City, Oregon, 97107",...,NaN,45.562081,-123.891503,United States,US,Oregon,Rockaway,5.99,2018-01-30 15:03:38,2022-04-23 06:37:53
2018-04-30 00:00:00,7986587,"(-122.34600236499995, 45.51805393300003)",KATU Channel 2,http://katu.com/news/local/rock-slide-temporar...,12711,2018-04-30 00:00:00,NaN,Historic Columbia River Highway small rockslide,A small rockslide disrupted traffic near Corbe...,"30701 E Historic Columbia River Hwy, Troutdale...",...,NaN,45.518054,-122.346002,United States,US,Oregon,Portland Troutdale,5.44,2018-06-06 13:41:55,2022-04-23 06:42:31
2018-05-11 00:00:00,7985597,"(-122.23650199999997, 44.10200500000008)",KLCC 89.7,http://klcc.org/post/crews-continue-removing-r...,11321,2018-05-11 00:00:00,NaN,Rock slide on Forest Road 19 Blocks Access to ...,8000 cubic yards (6116 cubic meters) of rock f...,"Forest Road 19 (Aufderheide Drive), Willamette...",...,NaN,44.102005,-122.236502,United States,US,Oregon,McKenzie Bridge,10.06,2018-05-11 16:07:51,2022-04-23 06:41:52
2018-12-25 00:00:00,7990447,"(-122.47909649199994, 45.39585714300006)",The Bull 98.7,https://www.987thebull.com/rock-slide-closes-h...,13294,2018-12-25 00:00:00,NaN,Highway 224 Rock fall near Tong Road,"A rock slide has closed Highway 224, near Tong...","17857-17907 OR-224, Damascus, Oregon, 97089",...,NaN,45.395857,-122.479096,United States,US,Oregon,Oregon City,11.35,2019-01-23 02:11:43,2022-04-23 06:45:09
2018-12-28 17:00:00,7982616,"(-122.16305099099998, 44.73584586900006)",KGW8,https://www.kgw.com/article/traffic/highway-22...,13297,2018-12-28 17:00:00,17:00,Highway 22 Rockslide west of Detroit,"300 yards of rocks slid town onto Highway 22, ...","OR-22, Detroit, Oregon, 97342",...,NaN,44.735846,-122.163051,United States,US,Oregon,Detroit Lake,14.77,2019-01-22 16:46:36,2022-04-23 06:39:56


### Get all SMAP, ESA, GPM and IMERG data files, sorted
 - ESA = Percent of Saturation Soil Moisture
 - SMAP = Volumetric soil moisture in cm3/cm3
 - GPM = Daily precipitation in mm
 - IMERG = 30 min precipitation in mm

In [9]:
data_dir = os.path.join(et.io.HOME,
                        'earth-analytics',
                        'data', 'capstone')
smap_files = sorted(glob(os.path.join(data_dir, 'smap_9km', '*.h5')))

# GPM daily files
gpm_files = sorted(glob(os.path.join(data_dir, 'gpm_westernUS', '*nc4')))

# GPM 30 min files
gpm_hires_files = sorted(glob(os.path.join(data_dir,
                                           'precip_imerg',
                                           'imerge',
                                           'glc', 'imerge*.csv')))

esa_files = sorted(glob(os.path.join(data_dir, 'esa_soil_moisture',
                                     '*ACTIVE*nc')))

# Print a sample as a sanity check
print(os.path.exists(smap_files[0]))
print(os.path.exists(gpm_files[0]))
print(os.path.exists(gpm_hires_files[0]))
print(os.path.exists(esa_files[0]))

True
True
True
True


### Load the EASE2 grid lon and lat datasets to subset SMAP data. 
- These can be found on the NSIDC website: https://nsidc.org/data/ease/tools#geo_data_files

> Brodzik, M. J., B. Billingsley, T. Haran, B. Raup, M. H. Savoie. 2012. EASE-Grid 2.0: Incremental but Significant Improvements for Earth-Gridded Data Sets. ISPRS International Journal of Geo-Information, 1(1):32-45, doi:10.3390/ijgi1010032. http://www.mdpi.com/2220-9964/1/1/32.

#### These are SMAP variables that can provide key information in characterizing landslides over Colorado

| Variable Name | Index | Units |
| ----------- | ----------- | ----------- |
| soil_moisture | 24 | cm<sup>3</sup>/cm<sup>3</sup> |
| radar_water_body_fraction | 15 | N/A | 
| vegetation_opacity | 46 | N/A |
| vegetation_water_content | 50 | kg/m<sup>2</sup> |

In [10]:
# Reshape to dimensions of the SMAP data above
sm_data, date = read_smap(smap_files[1], 24)

lats = np.fromfile(os.path.join(data_dir, 'smap_9km',
                                'EASE2_M09km.lats.3856x1624x1.double'),
                   dtype=np.float64).reshape(sm_data.shape)
lons = np.fromfile(os.path.join(data_dir, 'smap_9km',
                                'EASE2_M09km.lons.3856x1624x1.double'),
                   dtype=np.float64).reshape(sm_data.shape)
sm_data.shape

(1624, 3856)

### Read the SMAP, ESA CCI and GPM data 
- Two dataframes are generated
    1. Colocated to all the Landslide events
    2. Precipitation measurements going back 7 day from each landslide event and indexed to Landslide ID
        - GPM daily resolution precipitation

In [11]:
precip7d_date = []
precip7d = []
precip_accum = []
precip_max = []
smap_sm = []
smap_sm_7d = []
smap_wc = []
esa_sm = []
esa_sm_7d = []
landslide_date = []
landslide_id = []
landslide7d_id = []
landslide_cat = []
landslide_trig = []
landslide_sz = []
lat = []
lon = []
periods = 7
count_down = []

for i, ls_date in enumerate(glc_state_gt2015.index):

    # Get -7 days from the event
    glc_date = pd.date_range(ls_date, periods=periods,
                             freq='-1D').strftime('%Y%m%d')

    # Append landslide metadata
    landslide_date.append(ls_date)
    lat.append(glc_state_gt2015.latitude[i])
    lon.append(glc_state_gt2015.longitude[i])
    landslide_id.append(glc_state_gt2015.event_id[i])
    landslide_cat.append(glc_state_gt2015.landslide_category[i])
    landslide_trig.append(glc_state_gt2015.landslide_trigger[i])
    landslide_sz.append(glc_state_gt2015.landslide_size[i])

    # Take the +/- 0.3 deg mean around the Landslide event
    N_lat = glc_state_gt2015.latitude[i]+0.15
    S_lat = glc_state_gt2015.latitude[i]-0.15
    W_lon = glc_state_gt2015.longitude[i]-0.15
    E_lon = glc_state_gt2015.longitude[i]+0.15
    subset = (lats < N_lat) & (lats > S_lat) & (lons > W_lon) & (lons < E_lon)

    # Initialize data
    sm_max = []
    vegwc_max = []
    esa_mean = []
    precip = []
    countd = periods

    # loop over the 7 days
    for yyyymmdd in glc_date:

        # Find the SMAP file
        filesm = findfile(smap_files, yyyymmdd)
        # Retrieve the SMAP variables
        sm, time_t = read_smap(filesm[0], 24)
        vegwc, t = read_smap(filesm[0], 50)
        # Calculate the SMAP max
        sm_max.append(np.nanmax(sm[subset]))
        vegwc_max.append(np.nanmax(vegwc[subset]))

        # Get the Landslide location
        loc = (glc_state_gt2015.longitude[i], glc_state_gt2015.latitude[i])

        # Find the ESA soil moisture file
        file_esa = findfile(esa_files, yyyymmdd)
        # Get the nearest neighbor value of % soil moisture
        res_esa = nearestneighbor_ncdf(file_esa[0], 'sm', loc)
        # Replace negative values with NaN
        if res_esa < 0.0:
            esa_mean.append(np.nan)
        else:
            esa_mean.append(res_esa)

        # find the GPM file
        file_gpm = findfile(gpm_files, yyyymmdd)

        precip7d.append(nearestneighbor_ncdf(file_gpm[0],
                                             'precipitationCal', loc))
        landslide7d_id.append(glc_state_gt2015.event_id[i])
        # Append the date
        precip7d_date.append(yyyymmdd)

        precip.append(nearestneighbor_ncdf(file_gpm[0],
                                           'precipitationCal', loc))

        # Append countdown
        count_down.append(countd)
        countd -= 1

    # Append the summary values for the 7 day period
    smap_sm.append(np.nanmax(sm_max))
    smap_wc.append(np.nanmax(vegwc_max))
    esa_sm.append(np.nanmax(esa_mean))
    smap_sm_7d.extend(sm_max)
    esa_sm_7d.extend(esa_mean)

    # Filter for low or too high precipitation values
    if np.nansum(precip) < 0.4:
        precip_accum.append(np.nan)
        precip_max.append(np.nan)
    else:
        precip_accum.append(np.nansum(precip))
        precip_max.append(np.nanmax(precip))

# Create a soils and precip dataFrame
landslide_df = pd.DataFrame(smap_sm,
                            index=pd.to_datetime(landslide_date),
                            columns=['smap_sm'])
landslide_df['veg_water_content'] = smap_wc
landslide_df['esa_sm_percent'] = esa_sm
landslide_df['gpm_7day_accum_mm'] = precip_accum
landslide_df['gpm_7day_max_mm'] = precip_max

# Add the Landslide metadata
landslide_df['glc_lat'] = lat
landslide_df['glc_lon'] = lon
landslide_df['landslide_id'] = landslide_id
landslide_df['landslide_category'] = landslide_cat
landslide_df['landslide_trigger'] = landslide_trig
landslide_df['landslide_size'] = landslide_sz

# Create the 7day precipitation dataFrame
landslide_precip7d_df = pd.DataFrame(precip7d,
                                     index=pd.to_datetime(precip7d_date),
                                     columns=['gpm_precip_mm'])
landslide_precip7d_df['landslide_id'] = landslide7d_id
landslide_precip7d_df['smap_sm'] = smap_sm_7d
landslide_precip7d_df['esa_vol'] = esa_sm_7d


# Add the cumulative precipitation
accum = []
for i, data in landslide_precip7d_df.groupby("landslide_id"):
    res = data['gpm_precip_mm'].cumsum().values
    if res.max() > 0.4:
        accum.extend(np.flip(data['gpm_precip_mm'].cumsum().values))
    else:
        accum.extend([np.nan]*periods)

landslide_precip7d_df['gpm_7day_accum_mm'] = accum
landslide_precip7d_df['days-to-landslide'] = count_down
landslide_precip7d_df.index.names = ['date']

# Add the normalized precipitation
accum_norm = []
for i, data in landslide_precip7d_df.groupby("landslide_id"):
    res = data['gpm_7day_accum_mm']/data['gpm_7day_accum_mm'].mean()
    accum_norm.append(res.max())

landslide_df['gpm_7day_accum_norm'] = accum_norm

In [12]:
landslide_precip7d_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 497 entries, 2015-08-29 to 2018-12-22
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   gpm_precip_mm      497 non-null    float64
 1   landslide_id       497 non-null    int64  
 2   smap_sm            243 non-null    float32
 3   esa_vol            453 non-null    float64
 4   gpm_7day_accum_mm  497 non-null    float64
 5   days-to-landslide  497 non-null    int64  
dtypes: float32(1), float64(3), int64(2)
memory usage: 25.2 KB


In [13]:
landslide_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 71 entries, 2015-08-29 00:00:00 to 2018-12-28 17:00:00
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   smap_sm              71 non-null     float32
 1   veg_water_content    71 non-null     float32
 2   esa_sm_percent       70 non-null     float64
 3   gpm_7day_accum_mm    71 non-null     float64
 4   gpm_7day_max_mm      71 non-null     float64
 5   glc_lat              71 non-null     float64
 6   glc_lon              71 non-null     float64
 7   landslide_id         71 non-null     int64  
 8   landslide_category   71 non-null     object 
 9   landslide_trigger    71 non-null     object 
 10  landslide_size       71 non-null     object 
 11  gpm_7day_accum_norm  71 non-null     float64
dtypes: float32(2), float64(6), int64(1), object(3)
memory usage: 6.7+ KB


### Add the IMERGE 30min resolution precipitation to the landslide_df

#### First, read the IMERGE 30min data

In [14]:
imerg_30min = get_imerg_hires(gpm_hires_files, glc_state_gt2015)

/Users/jwitte/earth-analytics/data/capstone/precip_imerg/imerge/glc/imerge.2015.csv
/Users/jwitte/earth-analytics/data/capstone/precip_imerg/imerge/glc/imerge.2016.csv
/Users/jwitte/earth-analytics/data/capstone/precip_imerg/imerge/glc/imerge.2017.csv
/Users/jwitte/earth-analytics/data/capstone/precip_imerg/imerge/glc/imerge.2018.csv
/Users/jwitte/earth-analytics/data/capstone/precip_imerg/imerge/glc/imerge.2019.csv
/Users/jwitte/earth-analytics/data/capstone/precip_imerg/imerge/glc/imerge.2020.csv


#### Loop over the newly created landslide_df
- Add the sum and the max

In [15]:
imerg_7day_sum = []
imerg_7day_max = []
imerg_7day_accum = []
imerg_7day_date = []
imerg_7day_id = []

for i, ls_date in enumerate(landslide_df.index):

    glc_date = pd.date_range(ls_date, periods=7,
                             freq='-1D').strftime('%Y%m%d').to_list()

    # selecting rows based on condition
    rslt_df = imerg_30min[
                    (imerg_30min['id'] == landslide_df['landslide_id'][i]) &
                    imerg_30min['yyyymmdd'].isin(glc_date)
                    ]
    # first calculate the 7 day precipitation stats
    if rslt_df.shape[0] > 0:
        imerg_7day_sum.append(rslt_df['precipitation'].sum())
        imerg_7day_max.append(rslt_df['precipitation'].max())
        # append the 7 day daily values
        imerg_7day_accum.extend(rslt_df.precipitation.resample('D').sum())
        imerg_7day_date.extend(glc_date)
        imerg_7day_id.extend([landslide_df['landslide_id'][i]]*7)
    else:
        imerg_7day_sum.append(np.nan)
        imerg_7day_max.append(np.nan)
        imerg_7day_accum.extend([np.nan]*7)
        imerg_7day_date.extend(glc_date)
        imerg_7day_id.extend([landslide_df['landslide_id'][i]]*7)

landslide_df['imerg_7day_accum_mm'] = imerg_7day_sum
landslide_df['imerg_7day_max_mm'] = imerg_7day_max
landslide_df.index.name = 'date'

landslide_precip7d_df['imerg_precip_mm'] = imerg_7day_accum

In [16]:
landslide_precip7d_df

,gpm_precip_mm,landslide_id,smap_sm,esa_vol,gpm_7day_accum_mm,days-to-landslide,imerg_precip_mm
date,,,,,,,
2015-08-29,7.145266,7912,0.285622,26.420483,175.279051,7,NaN
2015-08-28,0.565224,7912,NaN,NaN,174.951981,6,NaN
2015-08-27,0.001086,7912,NaN,31.506886,160.905293,5,NaN
2015-08-26,0.005000,7912,0.213500,20.557873,135.678568,4,NaN
2015-08-25,0.000000,7912,NaN,17.890720,127.122822,3,NaN
...,...,...,...,...,...,...,...
2018-12-26,1.151209,13297,NaN,NaN,16.611134,5,NaN
2018-12-25,0.000088,13297,NaN,54.841618,1.952697,4,NaN
2018-12-24,14.658437,13297,0.411319,66.755066,1.952609,3,NaN


### Export the dataFrame to a csv file

In [17]:
# converting to CSV file
output_file = os.path.join('data','glc_smap_esa_gpm_2015-2020_'+state+'.csv')
landslide_df.to_csv(output_file)

output_7d_file = os.path.join('data',
                              'glc_smap_esa_gpm_2015-2020_7day_'+state+'.csv')
landslide_precip7d_df.to_csv(output_7d_file)